In [1]:
import scipy
import numpy as np
from scipy import signal
import pandas as pd
import glob
from scipy import interpolate
print('OK')

OK


In [2]:
ship_name='BLUE_JAY'

"""サンプリング周波数"""
fs_HMS=33
cut_filter=0.02 #カット周波数

"""各データセットのpath"""
HMS_path= 'J:/14000TEU/1 NYK BLUE_JAY/20180621+20190719/HMS/*.csv'#'G:/大量データ解析/データ格納庫/BLUE-JAY/HMS_33Hz/*.csv'
SIMS_path='J:/14000TEU/1 NYK BLUE_JAY/20180621+20190719/SIMS/*.csv'#'G:/大量データ解析/データ格納庫/BLUE-JAY/SIMS_10Hz/*.csv'
#VDR_path= 'J:/14000TEU/2 NYK IBIS/20180621+20190719/SIMS/*.csv'#'G:/大量データ解析/データ格納庫/BLUE-JAY/VDR/*.csv'
print(glob.glob(HMS_path)[0][len(HMS_path)-5+11:len(HMS_path)-5+11+10])
print(glob.glob(SIMS_path)[0][len(SIMS_path)-5+9:len(SIMS_path)-5+9+10])

2014121916
2016021604


In [6]:
"""書き込みcsvファイル作成"""
f1=open('G:/大量データ解析/データ格納庫/垂直縦曲げと水平縦曲げ/'+ship_name+'_垂直水平応力_FFT.csv',"w")
f1.write("year,month,day,time,kn,HDG,H1,theta1,lamda1,w1,H2,theta2,lambda2,w2,error,HbyJWA,TbyJWA,keido,ido")
f1.write('\n')
f2=open('G:/大量データ解析/データ格納庫/垂直縦曲げと水平縦曲げ/m0_'+ship_name+'_垂直水平応力_FFT.csv',"w")
f2.write("year,month,day,time,kn,HDG,H1,theta1,lamda1,w1,H2,theta2,lambda2,w2,error,HbyJWA,TbyJWA,keido,ido,verti,horizon")
f2.write('\n')

basis_data= pd.read_csv('G:/大量データ解析/データ格納庫/全データ/時間ずれ削除後/'+ship_name+'_by_FFT_全データ_時間ずれ削除後.csv', header=None,engine='python',skiprows=1,
                       usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])
basis=basis_data.values
print(np.shape(basis))
print(str('%04.0f'%(basis[400][0]))+str('%02.0f'%(basis[400][1]))+str('%02.0f'%(basis[400][2]))+str('%02.0f'%(basis[400][3])))

(6102, 19)
2017013115


In [7]:
"""各データの日付情報リスト作成"""
HMS_list=[str(i[len(HMS_path)-5:len(HMS_path)+16]) for i in glob.glob(HMS_path) if i[len(HMS_path)-5:len(HMS_path)+6] == 'RawStr_HMS_']
HMS_path_list=[i for i in glob.glob(HMS_path) if i[len(HMS_path)-5:len(HMS_path)+6] == 'RawStr_HMS_']
print(HMS_list[0])
print(HMS_path_list[0])

RawStr_HMS_2014121916
J:/14000TEU/1 NYK BLUE_JAY/20180621+20190719/HMS\RawStr_HMS_201412191658min_to_01min.csv


In [ ]:
"""実行部分"""
def Searching(csv_filename,listname):
    judge=csv_filename in listname
    if csv_filename in listname:
        return listname.index(csv_filename),judge
    return 0,judge

def FFT(time_series,fs):
    freq=np.linspace(0,fs,len(time_series))
    dt=1/fs
    fn=1/dt/2
    F=np.fft.fft(time_series)
    F=F/(len(time_series)/2)
    F[(freq>=fn)]=0#ナイキスト周波数以降はカット
    F[(freq<=cut_filter)]=0#ロ―カット
    return F

def Writing_basis(n_basis,f):
    #"year,month,day,time,kn,HDG,H1,theta1,lamda1,w1,H2,theta2,lambda2,w2,error,HbyJWA,TbyJWA,keido,ido
    for i in basis[n_basis]:
        f.write(str(i)+",")
    
class Calculation:
    def __init__(self,vertical,horizontal):
        self.Vertical=np.real(np.fft.ifft(FFT(vertical,fs_HMS)))*len(vertical)
        self.Horizontal=np.real(np.fft.ifft(FFT(horizontal,fs_HMS)))*len(horizontal)
        self.encounter_omega=np.arange(0,2.11,0.03)
        self.slide_size=330
        self.frame_size=2**16
        self.freq=0
        
    def Writing(self,x,f):
        S=interpolate.interp1d(self.freq,x,kind='nearest')
        X=S(self.encounter_omega)
        for i in range(len(self.encounter_omega)):
            f.write(str(X[i])+",")

    def Writing_m0(self,x,f):
        X=interpolate.interp1d(self.freq,x,kind='nearest')
        X1=X(self.encounter_omega)
        f.write(str(np.sum([(X1[k]+X1[k+1])*0.03/2 for k in range(70)]))+",")
    
    def Welch_ModifiedPeriodogram(self,X,Y):
        freq,csd=signal.csd(X,Y,fs_HMS,window='hann',nperseg=self.frame_size,noverlap=self.frame_size-self.slide_size,scaling='density')
        self.freq=freq*2*np.pi
        return csd/(2*np.pi)

def main():
    print("Start :"+str(len(basis))+"回")
    for n_basis in range(len(basis)):                            
        Date=str('%04.0f'%(basis[n_basis][0]))+str('%02.0f'%(basis[n_basis][1]))+str('%02.0f'%(basis[n_basis][2]))+str('%02.0f'%(basis[n_basis][3]))

        row, judge=Searching('RawStr_HMS_'+str(Date),HMS_list)
        if judge ==False:
            continue

        HMSdata=pd.read_csv(HMS_path_list[row],engine='python',usecols=[5,6],header=None,skiprows=1).dropna()#GMP,GMS
        HMSdata=HMSdata.values
        vertical=np.array(HMSdata[:,0]+HMSdata[:,1])/2
        horizontal=np.array(HMSdata[:,0]-HMSdata[:,1])/2
        
        DataSet=Calculation(vertical,horizontal)
        
        """計算&書き込み開始"""
        print(n_basis)
        Writing_basis(n_basis,f1)
        Writing_basis(n_basis,f2)
        """Power Spectrum Density"""
        XY = DataSet.Welch_ModifiedPeriodogram(DataSet.Vertical,DataSet.Vertical)
        DataSet.Writing(np.abs(XY),f1)
        DataSet.Writing_m0(np.abs(XY),f2)
        XY = DataSet.Welch_ModifiedPeriodogram(DataSet.Horizontal,DataSet.Horizontal)
        DataSet.Writing(np.abs(XY),f1)
        DataSet.Writing_m0(np.abs(XY),f2)
        """Cross Power SPectrum Density"""
        XY = DataSet.Welch_ModifiedPeriodogram(DataSet.Vertical,DataSet.Horizontal)
        DataSet.Writing(XY.real,f1)
        DataSet.Writing(XY.imag,f1)        
        f1.write('\n')
        f2.write('\n')
    print("End")
    f1.close()
    f2.close()

if __name__ == '__main__':
    main()

Start :6102回
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
